In [ ]:
import json
import boto3
import uuid


dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('social_media_post1')

def lambda_handler(event, context):
    # Process the incoming post
    post_text = event.get('text', '')
    hashtags = event.get('hashtags', [])
    
    # Generate a unique PostID
    post_id = str(uuid.uuid4())
    
    # Store the post in DynamoDB
    table.put_item(
        Item={
            'ID': post_id,
            'Hashtag': hashtags,
            'message': post_text
        }
    )
    
    # Retrieve all posts to calculate trending hashtags
    response = table.scan()
    items = response['Items']
    
    all_hashtags = []
    for item in items:
        all_hashtags.extend(item['Hashtag'])
    
    # Count the frequency of each hashtag (Manual Counter)
    hashtag_counts = {}
    for hashtag in all_hashtags:
        if hashtag in hashtag_counts:
            hashtag_counts[hashtag] += 1
        else:
            hashtag_counts[hashtag] = 1
    print(hashtag_counts.items())
    # Sort the hashtags by frequency in descending order and take the top 10
    trending_hashtags = sorted(hashtag_counts.items(), key=lambda item: item[1], reverse=True)[:10]
    
    return {
        'statusCode': 200,
        'body': json.dumps(trending_hashtags)
    }
